# Lab 01: Building a Transaction Data Pipeline

## What You'll Build

A real data integration pipeline that:
- Reads transaction data from PostgreSQL
- Filters transactions based on amount
- Writes to MySQL database AND CSV file

**Pipeline Flow:**
```
PostgreSQL → Peek → Filter → Transformer → MySQL
                                         └→ CSV File
```

**Learning Objectives:**
- Configure database connections
- Build multi-stage flows
- Route data to multiple destinations
- Define schemas and field mappings

---
## Part 1: Setup and Configuration

### Step 1.1: (Optional) Reinstall SDK

Only run this if you're starting fresh or having issues. **Skip if you just completed Lab 00.**

In [1]:
# Uncomment and run ONLY if needed:
# import sys
# !{sys.executable} -m pip uninstall ibm_watsonx_data_integration -y -q
# !{sys.executable} -m pip install ibm_watsonx_data_integration --force-reinstall -q
# print("✓ SDK reinstalled")

### Step 1.2: Import Libraries

In [2]:
from ibm_watsonx_data_integration import *
from ibm_watsonx_data_integration.common.auth import IAMAuthenticator
from ibm_watsonx_data_integration.services.datastage import *
from ibm_watsonx_data_integration.services.datastage.models.enums import SEQUENTIALFILE

print("✓ Imports complete")

✓ Imports complete


### Step 1.3: Set Credentials

**Use the same credentials from Lab 00.**

In [3]:
API_KEY = "qSOipvfdwXJjHSWxIgVd7u0d2V2WHsOR9A_V1SntP762"
PROJECT_ID = "a08157b8-d177-456f-82a3-7084576ff59e"

print("Credentials set (not displayed for security)")

Credentials set (not displayed for security)


### Step 1.4: Connect to Platform and Project

In [4]:
auth = IAMAuthenticator(api_key=API_KEY, base_auth_url="https://cloud.ibm.com")
platform = Platform(auth, base_api_url="https://api.ca-tor.dai.cloud.ibm.com")
project = platform.projects.get(guid=PROJECT_ID)

print(f"✓ Connected to project: {project.name}")

✓ Connected to project: Testing


### Step 1.5: Set Pipeline Parameters

In [5]:
from datetime import datetime

# Generate unique timestamp for CSV filename (prevents overwrite errors)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

FLOW_NAME = "TransactionsLowValue-SDK"
FILTER_CONDITION = "amount > 25"
MYSQL_TABLE = "processed_transactions"
CSV_FILENAME = f"low_value_transactions_{timestamp}.csv"

print("Pipeline Parameters:")
print(f"  Flow Name:    {FLOW_NAME}")
print(f"  Filter:       {FILTER_CONDITION}")
print(f"  MySQL Table:  {MYSQL_TABLE}")
print(f"  CSV File:     {CSV_FILENAME}")
print("\n✓ Setup complete!")

Pipeline Parameters:
  Flow Name:    TransactionsLowValue-SDK
  Filter:       amount > 25
  MySQL Table:  processed_transactions
  CSV File:     low_value_transactions_20251006_142326.csv

✓ Setup complete!


### (Optional) Step 1.6: Delete Existing Flow

**Only run this if you want to delete a previous version of this flow.**

Useful if you're re-running the lab and want to clean up.

In [6]:
# Uncomment to delete the existing flow before creating a new one:
# try:
#     duplicate = project.flows.get(name=FLOW_NAME)
#     project.delete_flow(duplicate)
#     print(f"✓ Deleted existing flow: {FLOW_NAME}")
# except Exception as e:
#     print(f"Flow not found or already deleted: {e}")

---
## Part 2: Build Pipeline Graph

Creates all stages, connects them, and defines schemas.

In [7]:
# Create flow
flow = project.create_flow(name=FLOW_NAME, environment=None, flow_type="datastage")
print(f"✓ Created flow: {FLOW_NAME}\n")

# Add PostgreSQL source
transactions_1 = flow.add_stage("PostgreSQL", "transactions_1")
transactions_1.configuration.runtime_column_propagation = False
transactions_1.configuration.table_name = "transactions"
transactions_1.configuration.connection.name = "PostgreSQL_conn"
transactions_1.configuration.connection.database = "cpd"
transactions_1.configuration.connection.defer_credentials = False
transactions_1.configuration.connection.hostname_or_ip_address = "52.116.198.152"
transactions_1.configuration.connection.password = "DataDuck!"
transactions_1.configuration.connection.port = "5432"
transactions_1.configuration.connection.proxy = False
transactions_1.configuration.connection.port_is_ssl_enabled = False
transactions_1.configuration.connection.username = "cpd"
print("✓ Added PostgreSQL source")

# Add Peek (monitoring)
peek_1 = flow.add_stage("Peek", "Peek_1")
peek_1.configuration.runtime_column_propagation = False
peek_1.configuration.outputlink_ordering_list = [{"link_label": "Output 1", "link_name": "Link_2"}]
print("✓ Added Peek_1")

# Add Filter
filter_1 = flow.add_stage("Filter", "Filter_1")
filter_1.configuration.show_coll_type = False
filter_1.configuration.show_part_type = True
filter_1.configuration.show_sort_options = False
filter_1.configuration.where_properties = [{"where": FILTER_CONDITION, "target": "0"}]
print(f"✓ Added Filter ({FILTER_CONDITION})")

# Add Transformer
transformer_1 = flow.add_stage("Transformer", "Transformer_1")
print("✓ Added Transformer")

# Add MySQL target
tm_ds_db_1_1 = flow.add_stage("MySQL", "TM_DS_DB_1_1")
tm_ds_db_1_1.configuration.column_metadata_change_propagation = False
tm_ds_db_1_1.configuration.output_acp_should_hide = False
tm_ds_db_1_1.configuration.schema_name = "TM_DS_DB_1"
tm_ds_db_1_1.configuration.show_coll_type = False
tm_ds_db_1_1.configuration.show_part_type = True
tm_ds_db_1_1.configuration.show_sort_options = False
tm_ds_db_1_1.configuration.table_name = MYSQL_TABLE
tm_ds_db_1_1.configuration.connection.name = "MySQL Legacy Financial DB"
tm_ds_db_1_1.configuration.connection.database = "TM_DS_DB_1"
tm_ds_db_1_1.configuration.connection.defer_credentials = "false"
tm_ds_db_1_1.configuration.connection.hostname_or_ip_address = "4d275b38-2eee-4b4d-8a88-cb022388e975.blijti4d0v0nkr55oei0.databases.appdomain.cloud"
tm_ds_db_1_1.configuration.connection.password = "eDGxvzFX7tK_"
tm_ds_db_1_1.configuration.connection.port = "32661"
tm_ds_db_1_1.configuration.connection.proxy = False
tm_ds_db_1_1.configuration.connection.port_is_ssl_enabled = True
tm_ds_db_1_1.configuration.connection.username = "TM_DS_USER"
print(f"✓ Added MySQL target ({MYSQL_TABLE})")

# Add second Peek
peek_2 = flow.add_stage("Peek", "Peek_2")
peek_2.configuration.outputlink_ordering_list = [{"link_label": "Output 1", "link_name": "Link_6"}]
print("✓ Added Peek_2")

# Add CSV target
sequential_file_1 = flow.add_stage("Sequential file", "Sequential_file_1")
sequential_file_1.configuration.file = [f"/ds-storage/{CSV_FILENAME}"]
sequential_file_1.configuration.first_line_is_column_names = SEQUENTIALFILE.FirstLineColumnNames.true
sequential_file_1.configuration.null_field_value = "'NULL'"
sequential_file_1.configuration.show_coll_type = True
sequential_file_1.configuration.show_part_type = False
sequential_file_1.configuration.show_sort_options = True
print(f"✓ Added CSV target ({CSV_FILENAME})\n")

# Connect stages
link_1 = transactions_1.connect_output_to(peek_1)
link_1.name = "Link_1"
link_2 = peek_1.connect_output_to(filter_1)
link_2.name = "Link_2"
link_3 = filter_1.connect_output_to(transformer_1)
link_3.name = "Link_3"
link_4 = transformer_1.connect_output_to(tm_ds_db_1_1)
link_4.name = "Link_4"
link_5 = transformer_1.connect_output_to(peek_2)
link_5.name = "Link_5"
link_6 = peek_2.connect_output_to(sequential_file_1)
link_6.name = "Link_6"
print("✓ Connected all stages\n")

# Define schemas
# Link 1: PostgreSQL → Peek_1
schema_1 = link_1.create_schema()
schema_1.add_field("INTEGER", "id")
schema_1.add_field("INTEGER", "account_id")
schema_1.add_field("VARCHAR", "timestamp").length(50)
schema_1.add_field("NUMERIC", "amount").length(10).scale(2)
schema_1.add_field("LONGVARCHAR", "location").length(1024)

# Link 2: Peek_1 → Filter
schema_2 = link_2.create_schema()
schema_2.add_field("INTEGER", "id").source("Link_1.id")
schema_2.add_field("INTEGER", "account_id").source("Link_1.account_id")
schema_2.add_field("VARCHAR", "timestamp").source("Link_1.timestamp").length(50)
schema_2.add_field("NUMERIC", "amount").source("Link_1.amount").length(10).scale(2)
schema_2.add_field("LONGVARCHAR", "location").source("Link_1.location").length(1024)

# Link 3: Filter → Transformer
schema_3 = link_3.create_schema()
schema_3.add_field("INTEGER", "id").source("Link_2.id")
schema_3.add_field("INTEGER", "account_id").source("Link_2.account_id")
schema_3.add_field("VARCHAR", "timestamp").source("Link_2.timestamp").length(50)
schema_3.add_field("NUMERIC", "amount").source("Link_2.amount").length(10).scale(2)
schema_3.add_field("LONGVARCHAR", "location").source("Link_2.location").length(1024)

# Link 4: Transformer → MySQL (removes id)
schema_4 = link_4.create_schema()
schema_4.add_field("INTEGER", "account_id").source("Link_3.account_id")
schema_4.add_field("VARCHAR", "timestamp").source("Link_3.timestamp").length(50)
schema_4.add_field("NUMERIC", "amount").source("Link_3.amount").length(10).scale(2)
schema_4.add_field("LONGVARCHAR", "location").source("Link_3.location").length(1024)

# Link 5: Transformer → Peek_2 (keeps all fields)
schema_5 = link_5.create_schema()
schema_5.add_field("INTEGER", "id").source("Link_3.id")
schema_5.add_field("INTEGER", "account_id").source("Link_3.account_id")
schema_5.add_field("VARCHAR", "timestamp").source("Link_3.timestamp").length(50)
schema_5.add_field("NUMERIC", "amount").source("Link_3.amount").length(10).scale(2)
schema_5.add_field("LONGVARCHAR", "location").source("Link_3.location").length(1024)

# Link 6: Peek_2 → CSV
schema_6 = link_6.create_schema()
schema_6.add_field("INTEGER", "id").source("Link_5.id")
schema_6.add_field("INTEGER", "account_id").source("Link_5.account_id")
schema_6.add_field("VARCHAR", "timestamp").source("Link_5.timestamp").length(50)
schema_6.add_field("NUMERIC", "amount").source("Link_5.amount").length(10).scale(2)
schema_6.add_field("LONGVARCHAR", "location").source("Link_5.location").length(1024)

print("✓ Defined schemas on all links")
print("\n✓ Pipeline graph complete!")

✓ Created flow: TransactionsLowValue-SDK

✓ Added PostgreSQL source
✓ Added Peek_1
✓ Added Filter (amount > 25)
✓ Added Transformer
✓ Added MySQL target (processed_transactions)
✓ Added Peek_2
✓ Added CSV target (low_value_transactions_20251006_142326.csv)

✓ Connected all stages

✓ Defined schemas on all links

✓ Pipeline graph complete!


---
## Part 3: Save and Execute

Saves the flow and creates a job to run it.

In [8]:
# Save flow
project.update_flow(flow)
print(f"✓ Flow saved: {FLOW_NAME}\n")

# Create and start job
job = project.create_job(name=f"{FLOW_NAME}_job", flow=flow)
print(f"✓ Job created: {FLOW_NAME}_job")

job_run = job.start(name=f"{FLOW_NAME} job run", description="Lab 01")
print(f"✓ Job started\n")

print("="*60)
print("SUCCESS! Pipeline is running.")
print("="*60)
print(f"\nWhat's happening now:")
print(f"  1. Reading from PostgreSQL (transactions table)")
print(f"  2. Filtering: {FILTER_CONDITION}")
print(f"  3. Writing to MySQL: {MYSQL_TABLE}")
print(f"  4. Writing to CSV: {CSV_FILENAME}")
print("\nNext: Go to the UI to verify (see lab guide)")
print("="*60)

✓ Flow saved: TransactionsLowValue-SDK



/Users/michaeldobson/Documents/Product Management/SDK_notebooks/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ca-tor.dai.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✓ Job created: TransactionsLowValue-SDK_job
✓ Job started

SUCCESS! Pipeline is running.

What's happening now:
  1. Reading from PostgreSQL (transactions table)
  2. Filtering: amount > 25
  3. Writing to MySQL: processed_transactions
  4. Writing to CSV: low_value_transactions_20251006_142326.csv

Next: Go to the UI to verify (see lab guide)


---
## Summary

### What You Built:
- **6-stage pipeline**: PostgreSQL → Peek → Filter → Transformer → MySQL + CSV
- **Database connections**: Configured PostgreSQL source and MySQL target
- **Data routing**: Single source to dual destinations
- **Schema definitions**: Complete field mappings across all links

### Key Concepts:
1. **Flow creation**: `project.create_flow()`
2. **Stage configuration**: Each stage has specific properties
3. **Link connections**: `stage1.connect_output_to(stage2)`
4. **Schema mapping**: `.source()` tracks data lineage
5. **Job execution**: `job.start()` runs the pipeline

### The Pattern:
```python
1. Create flow
2. Add stages
3. Connect stages
4. Define schemas
5. Save and execute
```

This pattern applies to **any** DataStage pipeline you build with the SDK.

---

## Next: Lab 02

You just built **one** pipeline manually.

Lab 02 shows you how to create **multiple pipelines** automatically using parameters and loops.

**Preview**: 3 pipeline variations (low/medium/high value) in under 5 minutes.

That's the power of SDK automation!